In [4]:
import requests
import time

# Base URL and credentials
QOGITA_API_URL = "https://api.qogita.com"
QOGITA_EMAIL = "jacek.budner@gmail.com"
QOGITA_PASSWORD = "JB100noga!"

# Step 1: Authentication
auth_response = requests.post(
    f"{QOGITA_API_URL}/auth/login/",
    json={"email": QOGITA_EMAIL, "password": QOGITA_PASSWORD}
).json()

if "accessToken" in auth_response:
    access_token = auth_response["accessToken"]
    headers = {"Authorization": f"Bearer {access_token}"}
    cart_qid = auth_response["user"]["activeCartQid"]

    print("Login successful.")
    print(f"Active Cart QID: {cart_qid}\n")

    # Step 2: Paginate through all results
    page = 1
    size = 5000  # Reasonable size per page
    total_count = 0

    while True:
        print(f"Fetching page {page}...")

        search_url = (
            f"{QOGITA_API_URL}/variants/search/?"
            f"page={page}&size={size}"
        )

        response = requests.get(url=search_url, headers=headers).json()

        results = response.get("results", [])
        if not results:
            print("No more results. Done.")
            break

        for variant in results:
            print(f"{variant['gtin']} | {variant['name']} | {variant['categoryName']} | {variant['brandName']} | "
                  f"{variant['price']} | {variant['inventory']} | {variant['imageUrl']} ")
            total_count += 1

        page += 1
        time.sleep(0.5)  # Sleep to avoid rate limits

    print(f"\n✅ Total products retrieved: {total_count}")

else:
    print("Login failed:", auth_response)


Login successful.
Active Cart QID: 69eb3544-57f1-4da4-b17d-e72a3c467c4b

Fetching page 1...
6291108730515 | Yara by Lattafa Perfumes Eau De Parfum 100ml 3.4 fl oz for Women | Eau De Parfum | Lattafa | 13.49 | 984 | https://static.prod.qogita.com/files/images/variants/TWttPXFk3XuSBGeaRmb2jj.jpg 
6290171002338 | Afnan 9 PM Eau De Parfum 100ml | Eau De Parfum | Afnan | 17.48 | 356 | https://static.prod.qogita.com/files/images/variants/VhLdYFBqgAFfk8ps9Jitu8.jpg 
6290360591421 | Yara Moi by Lattafa 100ml Eau De Perfume Spray | Eau De Parfum | Lattafa | 12.00 | 109 | https://static.prod.qogita.com/files/images/variants/k4ZCsCAXWYWEKeGcHKC3J2.jpg 
6291107456041 | LATTAFA FAKHAR by Lattafa Eau de Parfum Spray 3.4 oz | Eau De Parfum | Lattafa | 13.82 | 41 | https://static.prod.qogita.com/files/images/variants/7J8pi6Saxg6uHJrmQfXk3i.jpg 
6291108735411 | Men's Asad Eau De Parfum Spray 3.4 oz Fragrances | Eau De Parfum | Lattafa | 13.59 | 180 | https://static.prod.qogita.com/files/images/variants

KeyboardInterrupt: 

In [5]:
print(variant)

{'name': 'Narciso Rodriguez All Of Me Intense Eau De Parfum 90ml', 'gtin': '3423222108281', 'categoryName': 'Eau De Parfum', 'brandName': 'Narciso Rodriguez', 'inventory': 22, 'slug': 'narciso-rodriguez-all-of-me-intense-eau-de-parfum-90ml', 'fid': '4a3a1b5850214b2c8636bb4e6f644025', 'imageUrl': 'https://static.prod.qogita.com/files/images/variants/FoNQxBCyRNjjSFDifWGpcr.jpg', 'price': '73.74', 'priceCurrency': 'EUR', 'deals': [], 'position': 2000, 'isInStock': True, 'isWatchlisted': False, 'unit': None}


## VArianst add to cart 

In [ ]:
import requests
import time

# Credentials
QOGITA_API_URL = "https://api.qogita.com"
QOGITA_EMAIL = "jacek.budner@gmail.com"
QOGITA_PASSWORD = "JB100noga!"

# Step 1: Login
auth_response = requests.post(
    f"{QOGITA_API_URL}/auth/login/",
    json={"email": QOGITA_EMAIL, "password": QOGITA_PASSWORD}
).json()

if "accessToken" not in auth_response:
    print("❌ Login failed:", auth_response)
    exit()

access_token = auth_response["accessToken"]
headers = {"Authorization": f"Bearer {access_token}"}
cart_qid = auth_response["user"]["activeCartQid"]

print("✅ Authenticated successfully.")
print(f"🛒 Active Cart QID: {cart_qid}")

# Step 2: Get all variants (page by page)
page = 1
while True:
    search_response = requests.get(
        f"{QOGITA_API_URL}/variants/search/?page={page}&size=100",
        headers=headers
    ).json()

    results = search_response.get("results", [])
    if not results:
        print("❌ No more products found.")
        break  # Exit the loop when no products are left

    for variant in results:
        fid = variant.get("fid")
        slug = variant.get("slug")
        name = variant.get("name", "Unnamed Product")

        # Step 3: Get offers for each variant
        offers_url = f"{QOGITA_API_URL}/variants/{fid}/{slug}/offers/"

        try:
            offers_raw_response = requests.get(offers_url, headers=headers)

            # Handle rate-limiting (429)
            if offers_raw_response.status_code == 429:
                retry_after = int(offers_raw_response.headers.get("Retry-After", "5"))
                print(f"⏳ Rate limit hit. Sleeping for {retry_after} seconds...")
                time.sleep(retry_after)
                continue

            if not offers_raw_response.ok:
                print(f"❌ Failed to fetch offers for {name} | Status Code: {offers_raw_response.status_code}")
                print(f"🔎 Response text: {offers_raw_response.text[:200]}...")  # Show first 200 chars
                time.sleep(2)
                continue

            offers_response = offers_raw_response.json()

        except requests.exceptions.JSONDecodeError:
            print(f"❌ JSON decode error for {name}. Skipping.")
            time.sleep(2)
            continue

        offers = offers_response.get("results", []) if isinstance(offers_response, dict) else offers_response
        if not offers:
            print(f"❌ No offers found for {name}.")
            time.sleep(1)
            continue

        # Pick first offer
        offer = offers[0]
        offer_qid = offer["qid"]
        quantity = 100  # You can change this

        print(f"📦 Found Offer for {name}: {offer_qid} | Price: {offer['price']} | Inventory: {offer['inventory']}")

        # Step 4: Add to cart
        add_response = requests.post(
            url=f"{QOGITA_API_URL}/carts/{cart_qid}/lines/",
            json={"offerQid": offer_qid, "quantity": quantity},
            headers=headers
        )

        if add_response.ok:
            print(f"✅ Successfully added {quantity} units of {name} to cart.")
        else:
            print(f"❌ Failed to add {name} to cart:", add_response.text)

        time.sleep(1.5)  # Wait between offers to prevent rate limiting

    page += 1
    time.sleep(2)  # Wait before next page


✅ Authenticated successfully.
🛒 Active Cart QID: 69eb3544-57f1-4da4-b17d-e72a3c467c4b
📄 Offers Response Structure for Variant: Yara by Lattafa Perfumes Eau De Parfum 100ml 3.4 fl oz for Women
{'offers': [{'seller': 'GELD3O', 'price': '13.48', 'priceCurrency': 'EUR', 'inventory': 984, 'mov': '350.00', 'movCurrency': 'EUR', 'movProgress': '0.000', 'quantityInCart': 0, 'unit': 1, 'isTraceable': True, 'qid': 'f4966ec1-f053-4ae5-b6f0-c149058f5e9f'}, {'seller': 'O8V81R', 'price': '13.81', 'priceCurrency': 'EUR', 'inventory': 470, 'mov': '600.00', 'movCurrency': 'EUR', 'movProgress': '0.000', 'quantityInCart': 0, 'unit': 1, 'isTraceable': True, 'qid': 'c474f183-7291-4dc0-8da5-5a86a42999f8'}, {'seller': 'EMJVM7', 'price': '13.92', 'priceCurrency': 'EUR', 'inventory': 923, 'mov': '600.00', 'movCurrency': 'EUR', 'movProgress': '0.000', 'quantityInCart': 0, 'unit': 1, 'isTraceable': True, 'qid': 'fefd11b0-068f-498d-85bd-d99df0bfa43a'}, {'seller': 'KL6OOD', 'price': '14.36', 'priceCurrency': 'EUR'

JSONDecodeError: Expecting value: line 1 column 1 (char 0)